In [2]:
!rm /root/.kaggle
!mkdir /root/.kaggle
!cp /content/kaggle.json /root/.kaggle/

rm: cannot remove '/root/.kaggle': Is a directory
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [1]:
import sklearn
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
#import kaggle
#!kaggle datasets download -d datamunge/sign-language-mnist
#!unzip /content/sign-language-mnist.zip

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [6]:
dataset_train=pd.read_csv('D:\Working\signLnguage\sign_mnist_train.csv')
X_train=dataset_train.iloc[:,1:]
y_train=dataset_train.iloc[:,0]
X_train=np.array(X_train,dtype=np.float32)
y_train=np.array(y_train,dtype=np.float32)
X_train=X_train.reshape(X_train.shape[0],*(28,28,1))

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, test_size = 0.2, random_state = 12345)

In [12]:
model=tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(100, (4, 4), input_shape = (28,28,1), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size = (2, 2)))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units = 512, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 25, activation = 'softmax'))

In [13]:
model.compile(loss ='sparse_categorical_crossentropy', optimizer='adam' ,metrics =['accuracy'])


In [14]:
history = model.fit(X_train, y_train, batch_size = 512, epochs = 10, verbose = 1, validation_data = (X_validate, y_validate))

Epoch 1/10
43/43 [==============================] - 3s 63ms/step - loss: 23.1881 - accuracy: 0.0703 - val_loss: 1.9696 - val_accuracy: 0.4518
Epoch 2/10
43/43 [==============================] - 2s 57ms/step - loss: 2.0091 - accuracy: 0.3772 - val_loss: 0.9384 - val_accuracy: 0.7459
Epoch 3/10
43/43 [==============================] - 2s 58ms/step - loss: 1.2236 - accuracy: 0.6007 - val_loss: 0.4558 - val_accuracy: 0.8929
Epoch 4/10
43/43 [==============================] - 3s 62ms/step - loss: 0.7610 - accuracy: 0.7473 - val_loss: 0.1941 - val_accuracy: 0.9561
Epoch 5/10
43/43 [==============================] - 2s 58ms/step - loss: 0.4433 - accuracy: 0.8518 - val_loss: 0.0764 - val_accuracy: 0.9871
Epoch 6/10
43/43 [==============================] - 2s 58ms/step - loss: 0.2747 - accuracy: 0.9093 - val_loss: 0.0421 - val_accuracy: 0.9929
Epoch 7/10
43/43 [==============================] - 2s 57ms/step - loss: 0.1972 - accuracy: 0.9331 - val_loss: 0.0278 - val_accuracy: 0.9916
Epoch 8/10
4

In [17]:
model.save('/content/drive/MyDrive/sign-lang/sign-lang-version-1.h5')

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
model=tf.keras.models.load_model('D:\Working\signLnguage\sign-lang-version.h5')
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y' ]


In [3]:
cap=cv2.VideoCapture(0)
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
x=width//2
y=height//2
w,h=width//3,height//3

#writer=cv2.VideoWriter("MeBeingMad.mp4",cv2.VideoWriter_fourcc(*'DIVX'),25,(width,height))
while True:
    ret,frame=cap.read()
    
    cv2.rectangle(frame,pt1=(x,y),pt2=(x+w,y+h),color=(255,0,100),thickness=4)
    roi=frame[y:y+h,x:x+w]
    roi=cv2.resize(roi,(28,28))
    roi=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    roi=roi.reshape(1,28,28,1)
    res=model.predict_classes(roi)
    
    #print(newArea.shape)
#    writer.write(frame)
    cv2.putText(text=classes[res[0]],img=frame,org=(100,100),fontFace=cv2.FONT_HERSHEY_COMPLEX_SMALL,fontScale=5,color=(0,255,0))
    cv2.imshow("LiveFeed",frame)
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
cap.release()
#writer.release()
cv2.destroyAllWindows()

C:\Users\rrrdm\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
